In [1]:
import pandas as pd
import numpy as np
import os
import sys

_data_dir = 'data'
_preprocess_dir = 'data\\preprocess'
_predict_dir = 'data\\predict'

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

train_df = pd.read_csv(os.path.join(_preprocess_dir, 'train_features.csv'))

train_df.label = train_df.label.map({2: 1, 1: 1, 0: 0})

X = train_df.drop('label', axis=1)
y = train_df.label

print('X.shape: ', X.shape)
print('y.shape: ', y.shape)

zero_count = y.tolist().count(0)
one_count = y.tolist().count(1)
two_count = y.tolist().count(2)
print('All:  0: %f; 1: %f ; 2: %f' % (zero_count * 1.0 / len(y), one_count * 1.0 / len(y), two_count * 1.0 / len(y)))

X.shape:  (1269117, 38)
y.shape:  (1269117,)
All:  0: 0.937196; 1: 0.062804 ; 2: 0.000000


In [2]:
# DNN
from keras.layers import Input, BatchNormalization, concatenate, Flatten, Dropout, Dense,ReLU
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model, load_model
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint, TensorBoard

C:\Users\IfyourRtheone\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
print(y.shape)
print(X.shape)
X.head()

(1269117,)
(1269117, 38)


,user_id,id,gender,install_city_match,cargo_type,handling_type,truck_count,highway,lcl_cargo,truck_type_cargo_match_first,truck_type_cargo_match_second,truck_type_cargo_match_third,truck_type_cargo_match_fourth,truck_type_driver_match_first,truck_len_match,regular_subscribe_match,regular_search_3_match,regular_search_7_match,regular_search_14_match,regular_search_30_match,line_match,line_30days_match,line_60days_match,scan_cargo_level_3,scan_cargo_level_7,scan_cargo_level_14,scan_cargo_level_30,scan_cargo_level_avg,click_cargo_level_3,click_cargo_level_7,click_cargo_level_14,click_cargo_level_30,click_cargo_level_avg,call_cargo_level_3,call_cargo_level_7,call_cargo_level_14,call_cargo_level_30,call_cargo_level_avg
0,15090280313811400,390939613,1,False,-1.0,0.0,1.0,0.0,0.0,False,True,False,False,False,True,True,False,False,False,False,False,False,False,0,1,1,1,168,0,2,2,2,16,1,2,2,2,3
1,15090280313811400,390994195,1,False,0.0,0.0,3.0,0.0,0.0,False,False,True,False,False,True,True,False,False,False,False,True,True,True,0,1,1,1,168,0,2,2,2,16,1,2,2,2,3
2,15090280313811400,391142389,1,False,-1.0,0.0,1.0,0.0,0.0,False,True,False,False,False,True,True,False,False,False,False,False,False,False,0,1,1,1,168,0,2,2,2,16,1,2,2,2,3
3,15090280313811400,391165453,1,False,1.0,1.0,7.0,0.0,1.0,False,False,False,False,False,True,True,False,False,False,False,False,False,False,0,1,1,1,168,0,2,2,2,16,1,2,2,2,3
4,15090280313811400,391193650,1,False,-1.0,0.0,1.0,0.0,0.0,False,False,False,False,False,True,True,False,False,False,False,True,True,True,0,1,1,1,168,0,2,2,2,16,1,2,2,2,3


In [4]:
mean = np.mean(X['user_id'])
max = np.max(X['user_id'])
min = np.min(X['user_id'])
X['user_id'] = (X['user_id'] - min) / (max - min)
max = np.max(X['id'])
min = np.min(X['id'])
X['id'] = (X['id'] - min) / (max - min) 
X.head()

,user_id,id,gender,install_city_match,cargo_type,handling_type,truck_count,highway,lcl_cargo,truck_type_cargo_match_first,truck_type_cargo_match_second,truck_type_cargo_match_third,truck_type_cargo_match_fourth,truck_type_driver_match_first,truck_len_match,regular_subscribe_match,regular_search_3_match,regular_search_7_match,regular_search_14_match,regular_search_30_match,line_match,line_30days_match,line_60days_match,scan_cargo_level_3,scan_cargo_level_7,scan_cargo_level_14,scan_cargo_level_30,scan_cargo_level_avg,click_cargo_level_3,click_cargo_level_7,click_cargo_level_14,click_cargo_level_30,click_cargo_level_avg,call_cargo_level_3,call_cargo_level_7,call_cargo_level_14,call_cargo_level_30,call_cargo_level_avg
0,0.0,0.044878,1,False,-1.0,0.0,1.0,0.0,0.0,False,True,False,False,False,True,True,False,False,False,False,False,False,False,0,1,1,1,168,0,2,2,2,16,1,2,2,2,3
1,0.0,0.051492,1,False,0.0,0.0,3.0,0.0,0.0,False,False,True,False,False,True,True,False,False,False,False,True,True,True,0,1,1,1,168,0,2,2,2,16,1,2,2,2,3
2,0.0,0.069450,1,False,-1.0,0.0,1.0,0.0,0.0,False,True,False,False,False,True,True,False,False,False,False,False,False,False,0,1,1,1,168,0,2,2,2,16,1,2,2,2,3
3,0.0,0.072245,1,False,1.0,1.0,7.0,0.0,1.0,False,False,False,False,False,True,True,False,False,False,False,False,False,False,0,1,1,1,168,0,2,2,2,16,1,2,2,2,3
4,0.0,0.075662,1,False,-1.0,0.0,1.0,0.0,0.0,False,False,False,False,False,True,True,False,False,False,False,True,True,True,0,1,1,1,168,0,2,2,2,16,1,2,2,2,3


In [16]:
from keras import regularizers
def my_DNN(x):
  
    x = Dense(200, activation = "linear", name = "dense1",kernel_regularizer=regularizers.l2(0.01))(x)
    x = BatchNormalization(axis=-1, name = "BN1")(x)
    x = ReLU(name = "relu1")(x)
   # x = Dropout(0.5, name = "Dout1")(x)
    x = Dense(200,activation = "linear", name = "dense2",kernel_regularizer=regularizers.l2(0.01))(x)
    x = BatchNormalization(axis=-1, name = "BN2")(x)
    x = ReLU(name = "relu2")(x)
   # x = Dropout(0.5, name = "Dout2")(x)
    x = Dense(200, activation = "linear", name = "dense3")(x)
    x = BatchNormalization(axis=-1, name = "BN3")(x)
    x = ReLU(name = "relu3")(x)
   # x = Dropout(0.5, name = "Dout3")(x)
    x = Dense(200, activation = "linear", name = "dense4")(x)
    x = BatchNormalization(axis=-1, name = "BN4")(x)
    x = ReLU(name = "relu4")(x)
    output = Dense(2, activation = "sigmoid", name = "output")(x)
    return output

In [17]:
def my_DNN2(x):
    x = Dense(128, activation = "relu", name = "relu1")(x)
    x = Dense(128, activation = "relu", name = "relu2")(x)
    x = Dense(128, activation = "relu", name = "relu3")(x)
    output = Dense(2, activation= "softmax", name = "output")(x)
    return output

In [18]:
# def tensorSame(t1,t2):
#     aeqb = tf.equal(t1,t2)
#     aeqb_int = tf.to_int32(aeqb)
#     result = tf.equal(tf.reduce_sum(aeqb_int, axis=1),tf.reduce_sum(tf.ones_like(aeqb_int),axis = 1))
#     return result

In [19]:
# import tensorflow as tf
# from keras import backend as K
# def my_RMSE(y_true, y_pred):
#     with tf.Session() as sess:
#         loss_0 = 1.0 * K.sqrt(K.mean(K.square(y_true - y_pred)))
#         loss_1 = 2.0  * K.sqrt(K.mean(K.square(y_true - y_pred)))
#         loss_2 = 2.5  * K.sqrt(K.mean(K.square(y_true - y_pred)))
#         loss_total =tf.where(tensorSame(y_true, tensor2), loss_2, tf.where(tensorSame(y_true,tensor1), loss_1, loss_0))

#     return loss_total 

In [20]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

In [21]:
y = train_df.label
print(X.shape)
print(y.shape)

(1269117, 38)
(1269117,)


In [22]:
from keras import backend as K
input = Input(shape = (X.shape[1],), name = "input")
output = my_DNN(input)
#output = my_DNN2(input)
model =Model(input, output)
optimizer = Adam()
#model.compile(optimizer=optimizer, loss=my_RMSE, metrics=['accuracy', my_RMSE])
model.compile(optimizer=optimizer, loss=root_mean_squared_error, metrics=['accuracy', "categorical_crossentropy"])
y = to_categorical(y,num_classes=2)
tensorboard = TensorBoard(log_dir='tensorboard')
ckpt = ModelCheckpoint(filepath='checkpoints/ckpt1.hdf5', monitor='val_loss', save_best_only=True)

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, shuffle = True)

In [24]:
y_train.shape

(1205661, 2)

In [25]:
model.fit(X_train,y_train,batch_size=256,epochs=1,callbacks=[tensorboard, ckpt],validation_data=(X_test, y_test), shuffle = True)
model.save('model/my_DNN.h5')
model.summary()

Train on 1205661 samples, validate on 63456 samples
Epoch 1/1
1205661/1205661 [==============================] - 321s 266us/step - loss: 0.0888 - acc: 0.9365 - categorical_crossentropy: 0.6301 - val_loss: 0.0625 - val_acc: 0.9375 - val_categorical_crossentropy: 0.7141
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 38)                0         
_________________________________________________________________
dense1 (Dense)               (None, 200)               7800      
_________________________________________________________________
BN1 (BatchNormalization)     (None, 200)               800       
_________________________________________________________________
relu1 (ReLU)                 (None, 200)               0         
_________________________________________________________________
dense2 (Dense)               (None, 200)               40200     
_____

In [26]:
y_pred = np.argmax(model.predict(X_test), axis = 1)
y_test = np.argmax(y_test, axis = 1)
print(y_pred.shape)
print(y_test.shape)


(63456,)
(63456,)


In [27]:
import math

def calc_score(y_test, y_pred):
    assert(len(y_test) == len(y_pred))
    error = 0
    weight_dict = {
        0: 1,
        1: 2,
        2: 2.5
    }
    for y_t, y_p in zip(y_test, y_pred):
        error += (y_t - y_p) ** 2 * weight_dict[y_t]
    RMSE = math.sqrt(error / len(y_test))
    print('RMSE: ', RMSE)
    print('Score: ', 1 / (1 + RMSE))


In [28]:
calc_score(y_test, y_pred)

RMSE:  0.35350881473851764
Score:  0.7388204562178551


In [29]:
y1 = np.zeros((63456,))
calc_score(y_test,y1)

RMSE:  0.35350881473851764
Score:  0.7388204562178551
